In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlretrieve

In [2]:
import openpyxl

In [6]:
from selenium import webdriver
import time
import datetime

In [478]:
keyword = input("검색어를 입력해주세요: ")
#여성스포츠 "여성스포츠"

try:
    wb = openpyxl.load_workbook("2000_여성스포츠_뉴스.xlsx") #존재하는 파일에다가 새로운 데이터 누적하여 입력
    sheet = wb.active
    print("불러오기 완료")
except:
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.append([ "본문"])
    print("새로운 파일을 만들었습니다")

검색어를 입력해주세요: 여성스포츠 "여성스포츠"
새로운 파일을 만들었습니다


In [479]:
driver = webdriver.Chrome('./chromedriver')
driver.get('https://news.naver.com/')

In [480]:
# 검색창 : input.text_index
# 검색버튼 : button.btn_search_lnb
time.sleep(3)
search = driver.find_element_by_css_selector('input.text_index')
search.send_keys(keyword)
button = driver.find_element_by_css_selector('button.btn_search_lnb')
button.click()

In [481]:
driver.switch_to.window(driver.window_handles[1])
print("Current Page Title is : %s" %driver.title)

Current Page Title is : 여성스포츠 "여성스포츠" : 네이버 뉴스검색


In [483]:
# 검색옵션 : a#search_option_button.bt_option
# 날짜 : div._search_option_period_
# 기간(이전) : input#news_input_period_begin.input_txt._input_text_
# 기간(이후) : div.input_box._box_max_
# 적용 : div._search_option_period_ button._btn_submit_

# 검색옵션 열기
time.sleep(3)
search_opt_box = driver.find_element_by_xpath('//*[@id="search_option_button"]')
search_opt_box.click()
time.sleep(1)

tablist_box = driver.find_element_by_xpath('//div[@class="snb_inner"]/ul[@role="tablist" and @class="option_menu"]')

tablist_elem_list = tablist_box.find_elements_by_xpath('./li[@role="presentation"]')
press_box = [t for t in tablist_elem_list if t.text == '기간'][0].find_element_by_xpath('./a')
press_box.click()

In [484]:
date_frame = driver.find_element_by_xpath('.//div[@class="snb_itembox type_news _search_option_period_"]')

start_dt = datetime.datetime.strptime('2000.01.01', '%Y.%m.%d')
last_dt = datetime.datetime.strptime('2000.12.31', '%Y.%m.%d')
ds = start_dt.strftime('%Y.%m.%d')
de = last_dt.strftime('%Y.%m.%d')
min_date = driver.find_element_by_css_selector('input#news_input_period_begin.input_txt._input_text_')
min_date.send_keys(ds)
max_date = driver.find_element_by_css_selector('input#news_input_period_end.input_txt._input_text_')
max_date.send_keys(de)
button = driver.find_element_by_css_selector('div._search_option_period_ button._btn_submit_')
button.click()

In [485]:
while True:
    time.sleep(1)
    articles = driver.find_elements_by_css_selector('div.news_area')
    time.sleep(1)
    if len(articles) == 10:
        for article in articles:
            #title = article.find_element_by_css_selector('a.news_tit').text
            #dt = article.find_element_by_css_selector('span.info').text
            driver.switch_to.window(driver.window_handles[1])
            url = article.find_element_by_css_selector('a.news_tit')
            url = url.get_attribute('href')
            
            try:

                #open new tab
                driver.execute_script("window.open('');")
                driver.switch_to.window(driver.window_handles[2])
                driver.get(url)

                time.sleep(1)

                ## 기사본문: div#newsEndContents  //  div#articleBodyContents
                try:
                    content = driver.find_element_by_css_selector('div#newsEndContents').text
                except:
                    try:
                        content = driver.find_element_by_css_selector('div.article_body').text

                    except:
                        try:
                            content = driver.find_element_by_css_selector('div#article_txt').text
                        except:
                            content = ' '
                            pass
                sheet.append([ content])
                # close new tab
                driver.close()

                try:
                    driver.switch_to.window(driver.window_handles[2])
                    driver.close()
                except:
                    pass

                time.sleep(1)
                # naver news tab
                driver.switch_to.window(driver.window_handles[1])
            
            except:
                try:
                    driver.switch_to.window(driver.window_handles[2])
                    driver.close()
                except:
                    pass
            
                time.sleep(1)
                driver.switch_to.window(driver.window_handles[1])
                    
        
        page_bar = driver.find_elements_by_css_selector('div.sc_page > *')
        page_bar[2].click()
            
    else:
        for article in articles:
            #title = article.find_element_by_css_selector('a.news_tit').text
            #dt = article.find_element_by_css_selector('span.info').text
            driver.switch_to.window(driver.window_handles[1])
            url = article.find_element_by_css_selector('a.news_tit')
            url = url.get_attribute('href')

            #open new tab
            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[2])
            driver.get(url)

            time.sleep(1)

            ## 기사본문: div#newsEndContents  //  div#articleBodyContents
            try:
                content = driver.find_element_by_css_selector('div#newsEndContents').text
            except:
                try:
                    content = driver.find_element_by_css_selector('div.article_body').text
                except:
                    try:
                        content = driver.find_element_by_css_selector('div#article_txt').text
                    except:
                        content = ' '
                        pass
            sheet.append([content])
            # close new tab
            driver.close()
            
            try:
                driver.switch_to.window(driver.window_handles[2])
                driver.close()
            except:
                pass
            
            time.sleep(1)
            # naver news tab
            driver.switch_to.window(driver.window_handles[1])
        break

WebDriverException: Message: chrome not reachable
  (Session info: chrome=88.0.4324.190)


In [476]:
wb.save("2000_여성스포츠_뉴스.xlsx")
driver.close()
driver.switch_to.window(driver.window_handles[0])

WebDriverException: Message: chrome not reachable
  (Session info: chrome=88.0.4324.190)


In [183]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [486]:
wb.save("2000_여성스포츠_뉴스.xlsx")

In [ ]:
//*[@class="btn" and @aria-pressed="true"]